In [ ]:
import cv2
import numpy as np
from keras.models import load_model # type: ignore
from keras.applications.imagenet_utils import preprocess_input # type: ignore
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:

class_names = [
    "Speed limit 20", "Speed limit 30", "Speed limit 50", "Speed limit 60",
    "Speed limit 70", "Speed limit 80", "End of speed limit 80", "Speed limit 100",
    "Speed limit 120", "No passing", "No passing for trucks", "Right-of-way at intersection",
    "Priority road", "Yield", "Stop", "No vehicles", "Vehicles over 3.5t prohibited",
    "No entry", "General caution", "Dangerous curve left", "Dangerous curve right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right",
    "Road work", "Traffic signals", "Pedestrians", "Children crossing",
    "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing",
    "End of all restrictions", "Turn right ahead", "Turn left ahead",
    "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout", "End of no passing", "End of no passing for trucks"
]


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8_fine_tuned_v01.pt")

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    annotated_frame = results[0].plot()

    cv2.imshow("Traffic Sign Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8_fine_tuned_v01.pt")

In [4]:

image_folder = "GTSDB/images/test"
labels_folder = "GTSDB/labels/test"

output_file = "v1_test_pred.txt"

with open(output_file, "w") as f:

    for img_name in os.listdir(image_folder):
        if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(image_folder, img_name)
            results = model.predict(source=img_path, save=False, conf=0.25, verbose=False)

            class_ids = results[0].boxes.cls.tolist()

            f.write(f"{img_name} " + " ".join(str(int(cls_id)) for cls_id in class_ids) + "\n")


In [5]:
with open(output_file, "r") as f:
    lines = f.readlines()

In [6]:
pred = []
for line in lines:
    parts = line.strip().split(' ')
    img_name = parts[0]
    if len(parts) > 1:
        class_ids = [int(cls_id) for cls_id in parts[1:]]
    else:
        class_ids = [99]
    if class_ids:
        pred.append((img_name, class_ids))

In [7]:
actual = []
for file in os.listdir(labels_folder):
    with open(os.path.join(labels_folder, file), "r") as f:
        content = f.readlines()
        content = [line.strip() for line in content if line.strip()]
        if content:
            class_id = []
            for line in content:
                class_id.append(int(line.split()[0]))
            actual.append((file, class_id))   

In [8]:
sum = 0
count = 0

for i in range(len(actual)):
    count += (max(len(actual[i][1]), len(pred[i][1])))

for i in range(len(actual)):
    for j in pred[i][1]:
        if j in actual[i][1]:
            sum += 1
accuracy = sum / count
accuracy

0.6475409836065574